In [1]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt

## Reading in the data and preparing DataFrames

#### MID

In [2]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
#print(mid.describe())
# #bring the time data into a new df:
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column

#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)
print(time)
time_sliced = time.iloc[:,11:].fillna(0)
print(time_sliced)

#convert elapsed times in time-dataframe back to a timestamp
def h2_time(df1,df2):
    startdate = pd.to_datetime(df1.iloc[2,0],format="%Y/%m/%d %H:%M:%S.%f")
    dic= {}
    itr = 0
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    key = "Timestamp, mass {}".format(masses[itr])
    for label,content in df2.items():
        key = "Timestamp, mass {}".format(masses[itr])
        #.strftime('%d-%m-%Y %H:%M:%S') not used
        for row in content:
            dic.setdefault(key,[]).append((startdate+dt.timedelta(milliseconds=row*1000)))
        itr += 1
    new_df = pd.DataFrame.from_dict(dic)
    return(new_df)

new_df = h2_time(time,time_sliced)
new_df

               mass 2.0_time            mass 4.0_time  \
2    2022/12/13 17:34:18.708  2022/12/13 17:34:18.958   
3    2022/12/13 17:34:25.033  2022/12/13 17:34:25.283   
4    2022/12/13 17:34:31.358  2022/12/13 17:34:31.608   
5    2022/12/13 17:34:37.683  2022/12/13 17:34:37.933   
6    2022/12/13 17:34:44.008  2022/12/13 17:34:44.258   
..                       ...                      ...   
341  2022/12/13 18:10:02.883  2022/12/13 18:10:03.133   
342  2022/12/13 18:10:09.208  2022/12/13 18:10:09.458   
343  2022/12/13 18:10:15.533  2022/12/13 18:10:15.783   
344  2022/12/13 18:10:21.858  2022/12/13 18:10:22.108   
345  2022/12/13 18:10:28.183  2022/12/13 18:10:28.433   

               mass 5.5_time           mass 12.0_time  \
2    2022/12/13 17:34:19.208  2022/12/13 17:34:19.458   
3    2022/12/13 17:34:25.533  2022/12/13 17:34:25.783   
4    2022/12/13 17:34:31.858  2022/12/13 17:34:32.108   
5    2022/12/13 17:34:38.183  2022/12/13 17:34:38.433   
6    2022/12/13 17:34:44.508  

,"Timestamp, mass 2","Timestamp, mass 4","Timestamp, mass 5.5","Timestamp, mass 12","Timestamp, mass 15","Timestamp, mass 16","Timestamp, mass 18","Timestamp, mass 28","Timestamp, mass 32","Timestamp, mass 40","Timestamp, mass 44"
0,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358,2022-12-13 17:34:31.358
1,2022-12-13 17:34:37.683,2022-12-13 17:34:37.683,2022-12-13 17:34:37.683,2022-12-13 17:34:37.683,2022-12-13 17:34:37.683,2022-12-13 17:34:37.683,2022-12-13 17:34:37.683,2022-12-13 17:34:37.683,2022-12-13 17:34:37.684,2022-12-13 17:34:37.683,2022-12-13 17:34:31.358
2,2022-12-13 17:34:44.008,2022-12-13 17:34:44.008,2022-12-13 17:34:44.008,2022-12-13 17:34:44.008,2022-12-13 17:34:44.008,2022-12-13 17:34:44.009,2022-12-13 17:34:44.008,2022-12-13 17:34:44.008,2022-12-13 17:34:44.009,2022-12-13 17:34:44.008,2022-12-13 17:34:31.358
3,2022-12-13 17:34:50.333,2022-12-13 17:34:50.333,2022-12-13 17:34:50.333,2022-12-13 17:34:50.333,2022-12-13 17:34:50.333,2022-12-13 17:34:50.334,2022-12-13 17:34:50.333,2022-12-13 17:34:50.333,2022-12-13 17:34:50.334,2022-12-13 17:34:50.333,2022-12-13 17:34:31.358
4,2022-12-13 17:34:56.658,2022-12-13 17:34:56.658,2022-12-13 17:34:56.658,2022-12-13 17:34:56.658,2022-12-13 17:34:56.658,2022-12-13 17:34:56.659,2022-12-13 17:34:56.658,2022-12-13 17:34:56.658,2022-12-13 17:34:56.659,2022-12-13 17:34:56.658,2022-12-13 17:34:31.358
...,...,...,...,...,...,...,...,...,...,...,...
339,2022-12-13 18:10:15.533,2022-12-13 18:10:15.533,2022-12-13 18:10:15.533,2022-12-13 18:10:15.533,2022-12-13 18:10:15.533,2022-12-13 18:10:15.534,2022-12-13 18:10:15.533,2022-12-13 18:10:15.533,2022-12-13 18:10:15.534,2022-12-13 18:10:15.533,2022-12-13 17:34:31.358
340,2022-12-13 18:10:21.858,2022-12-13 18:10:21.858,2022-12-13 18:10:21.858,2022-12-13 18:10:21.858,2022-12-13 18:10:21.858,2022-12-13 18:10:21.859,2022-12-13 18:10:21.858,2022-12-13 18:10:21.858,2022-12-13 18:10:21.859,2022-12-13 18:10:21.858,2022-12-13 17:34:31.358
341,2022-12-13 18:10:28.183,2022-12-13 18:10:28.183,2022-12-13 18:10:28.183,2022-12-13 18:10:28.183,2022-12-13 18:10:28.183,2022-12-13 18:10:28.184,2022-12-13 18:10:28.183,2022-12-13 18:10:28.183,2022-12-13 18:10:28.184,2022-12-13 18:10:28.183,2022-12-13 17:34:31.358
342,2022-12-13 18:10:34.508,2022-12-13 18:10:34.508,2022-12-13 18:10:34.508,2022-12-13 18:10:34.508,2022-12-13 18:10:34.508,2022-12-13 18:10:34.509,2022-12-13 18:10:34.508,2022-12-13 18:10:34.508,2022-12-13 18:10:34.509,2022-12-13 18:10:34.508,2022-12-13 17:34:31.358


#### Vaclogger

In [3]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()

#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
0,NaN,2022-12-13 17:31:14,0.0,-0.046295,2.540000e-09,1.640000e-09,4.996000e-09,4.919000e-07,0.35246,197.0,-5.163,4288.086,NaN,NaN
1,NaN,2022-12-13 17:31:21,7.0,-0.047451,2.900000e-09,1.680000e-09,4.996000e-09,4.919000e-07,0.35245,197.0,-5.275,4288.198,NaN,NaN
2,NaN,2022-12-13 17:31:28,14.0,-0.046614,2.900000e-09,1.640000e-09,4.996000e-09,4.919000e-07,0.35244,197.0,-5.050,4286.403,NaN,NaN
3,NaN,2022-12-13 17:31:35,21.0,-0.050953,2.400000e-09,1.860000e-09,4.996000e-09,4.733000e-07,0.35245,196.0,-5.275,4286.739,NaN,NaN
4,NaN,2022-12-13 17:31:42,28.0,-0.046468,1.140000e-09,6.030000e-10,4.996000e-09,4.688000e-07,0.35244,196.0,-5.163,4286.515,NaN,NaN


#### HiVolta log

In [4]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")

#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["I_em"]=[element * 1e-6 for element in hv["IMon2"]]

# hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["I_em"]=abs(hv["I_em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding a datetime column
datetime = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
datetime = datetime.dt.strftime('%d-%m-%Y %H:%M:%S')
datetime = pd.to_datetime(datetime, format='%d-%m-%Y %H:%M:%S')
hv.insert(0,"datetime",datetime)
    
#making the column into a datetime object
hv_timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(hv_timestamps-hv_timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)

hv.head()



,datetime,Date,Elapsed time,Time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,...,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,Unnamed: 19,hv_grid,I_em
0,2022-12-13 17:30:03,13/12/2022,0.000,17:30:03.498,0.06,0.1,0.18,0.18,0.0,0.0,...,0.0129,0.0105,0.0167,-0.0142,-0.0258,-0.0100,NaN,NaN,1.000000e-10,6.200000e-09
1,2022-12-13 17:30:04,13/12/2022,1.178,17:30:04.676,0.06,0.1,0.18,0.18,0.0,0.0,...,0.0129,0.0105,0.0164,-0.0142,-0.0258,-0.0100,NaN,NaN,1.000000e-10,6.200000e-09
2,2022-12-13 17:30:05,13/12/2022,2.366,17:30:05.864,0.06,0.1,0.18,0.18,0.0,0.0,...,0.0129,0.0105,0.0101,-0.0142,-0.0258,-0.0100,NaN,NaN,1.000000e-10,6.200000e-09
3,2022-12-13 17:30:07,13/12/2022,3.548,17:30:07.046,0.06,0.1,0.18,0.18,0.0,0.0,...,0.0129,0.0105,0.0171,-0.0142,-0.0258,-0.0100,NaN,NaN,1.000000e-10,6.200000e-09
4,2022-12-13 17:31:58,13/12/2022,114.845,17:31:58.343,0.06,0.1,0.14,0.16,0.0,0.0,...,0.0165,0.0082,0.0124,-0.0143,-0.0262,-0.0099,NaN,NaN,2.000000e-10,5.700000e-09


## Data processing

#### Comments

In [5]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan]


In [6]:
#print hv comments
print(pd.unique(hv["Comment"]))
hv_comm = hv.loc[:,["datetime","Comment"]]
hv_comm.dropna(inplace=True)
hv_comm.index

[nan 'no emission' 'emission on' 'minimal emission current'
 'grid repelling' 'grid transparent' 'ega turned' 'grid repelling again'
 'no electron activity']


Int64Index([  17,   18,   19,   20,   50,   51,   52,   53,  339,  340,  341,
             342,  652,  653,  654,  655,  660,  661,  662,  663,  776,  777,
             778,  779,  834,  835,  836,  837, 1359, 1360, 1361, 1362, 1857,
            1858, 1859, 1860],
           dtype='int64')

#### CernOx temperature R-T conversion

In [7]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]

#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


#### Data preparation for calculating desorption yield from first ESD spike (w/o recycling)

In [8]:
#Filtering background signal

#Find the timestamp where emission was turned on and off
#print("Em on:",hv.loc[hv["Comment"]=="emission on"])
#print("Grid repelling:", hv.loc[hv["Comment"]=="grid repelling"])
x=pd.Timestamp(2022, 12, 13, 17,38,53)
y=pd.Timestamp(2022, 12, 13, 17, 45, 21)

#then find the time value closest to it in the MID scan file 
#print("\n","Em on in MID file:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
#print("\n","Grid repelling in MID file:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])

#take a mass with a signal known to be in the electronics background
elec_bck = np.mean(mid["mass 5.5_value"])
print("electr_ bck:",f"{elec_bck}")

#apply electronic background correction to the mass 2.0 data
mid["mass 2, corr"] = mid["mass 2.0_value"] - elec_bck

#apply base pressure correction to mass 2.0 data
dyn_bck = np.mean(mid["mass 2, corr"][0:44])
print("dyn_ bck:",f"{dyn_bck}")

mid["mass 2, corr"] = mid["mass 2, corr"] - dyn_bck

#remove values below 0:
mid["mass 2, corr"].clip(0, inplace=True)

#From the signal, get the partial pressures 
h2_sens = 2.13e-2 #A/mbar
h2_equiv = 2.49 #H2 equivalent pressures from N2 reading

#account for the thermal transpiration factor
T2 = 293.5
T1 = 4.2
T_coef = np.sqrt(T2/T1)

#apply correction factors 
p_h2 = (mid["mass 2, corr"]*h2_equiv)/(h2_sens*T_coef) 
mid.insert(22,"p_h2",p_h2)
pd.set_option('display.max_rows', 100) 
print(p_h2)


#baseline and emission pp for partial pressure delta
em_h2 = mid["p_h2"][40:75] #hardcoded values for "esd on"
base_h2 = mid["p_h2"][0:40] #hardcoded values for "esd off"
print(em_h2)

electr_ bck: 1.875755813953489e-14
dyn_ bck: 2.0234387896405914e-11
2      0.000000e+00
3      0.000000e+00
4      0.000000e+00
5      0.000000e+00
6      0.000000e+00
           ...     
341    3.296859e-12
342    2.710918e-12
343    2.778042e-12
344    3.666044e-12
345    1.966954e-12
Name: mass 2, corr, Length: 344, dtype: float64
42    2.078828e-12
43    2.643793e-12
44    3.860426e-12
45    3.289867e-12
46    1.533174e-11
47    1.069315e-11
48    8.581521e-12
49    8.100462e-12
50    8.799676e-12
51    6.176224e-12
52    7.492145e-12
53    5.379120e-12
54    6.987313e-12
55    4.935818e-12
56    4.049214e-12
57    7.025070e-12
58    6.048967e-12
59    4.499508e-12
60    4.748428e-12
61    3.733169e-12
62    3.281477e-12
63    3.191977e-12
64    2.778042e-12
65    1.020217e-12
66    2.996197e-12
67    3.853434e-12
68    3.289867e-12
69    1.726424e-12
70    1.418770e-12
71    1.178240e-12
72    2.185108e-12
73    9.293195e-13
74    2.710918e-12
75    1.501277e-12
76    3.403140e-12

In [9]:
#need to resample to find the correct delta p and corresponding emission currents!
#reindex hv to match the sampling rate of MID  
# Convert the 'time' column to datetime index for both dataframes
hv.index = (pd.to_datetime(hv["datetime"]).dt.round(freq="S"))
mid.index = (pd.to_datetime(mid["mass 2.0_time"]).dt.round(freq="S"))

# Reindex df1 to match the datetime index of df2
hv = hv.reindex(mid.index, method='nearest', limit=1)

# Now df1 and df2 have the same datetime index and can be joined or aligned
# by joining on the index
aligned_df = mid.merge(hv,left_index=True, right_index=True, how="inner")

#replace datetime index with normal index
aligned_df.reset_index(drop=True,inplace=True)
aligned_df.head()

,mass 2.0_time,mass 2.0_value,mass 4.0_time,mass 4.0_value,mass 5.5_time,mass 5.5_value,mass 12.0_time,mass 12.0_value,mass 15.0_time,mass 15.0_value,...,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,Unnamed: 19,hv_grid,I_em
0,2022/12/13 17:34:18.708,1.921830e-11,2022/12/13 17:34:18.958,9.218000e-13,2022/12/13 17:34:19.208,5.800000e-14,2022/12/13 17:34:19.458,1.746000e-13,2022/12/13 17:34:19.708,9.400000e-14,...,0.0084,0.01,0.0087,-0.0156,-0.0274,-0.0109,NaN,NaN,3.000000e-10,5.600000e-09
1,2022/12/13 17:34:25.033,1.945420e-11,2022/12/13 17:34:25.283,9.547000e-13,2022/12/13 17:34:25.533,5.770000e-14,2022/12/13 17:34:25.783,1.823000e-13,2022/12/13 17:34:26.033,9.930000e-14,...,0.0084,0.01,0.0087,-0.0156,-0.0274,-0.0110,NaN,NaN,3.000000e-10,6.400000e-09
2,2022/12/13 17:34:31.358,1.969180e-11,2022/12/13 17:34:31.608,9.517000e-13,2022/12/13 17:34:31.858,6.270000e-14,2022/12/13 17:34:32.108,1.666000e-13,2022/12/13 17:34:32.358,1.068000e-13,...,0.0084,0.01,0.0087,-0.0156,-0.0274,-0.0110,NaN,NaN,3.000000e-10,5.800000e-09
3,2022/12/13 17:34:37.683,1.985630e-11,2022/12/13 17:34:37.933,9.632000e-13,2022/12/13 17:34:38.183,1.580000e-14,2022/12/13 17:34:38.433,1.768000e-13,2022/12/13 17:34:38.683,1.148000e-13,...,0.0084,0.01,0.0087,-0.0156,-0.0274,-0.0110,NaN,NaN,3.000000e-10,5.800000e-09
4,2022/12/13 17:34:44.008,1.973530e-11,2022/12/13 17:34:44.258,9.713000e-13,2022/12/13 17:34:44.508,1.850000e-14,2022/12/13 17:34:44.758,1.684000e-13,2022/12/13 17:34:45.008,9.780000e-14,...,0.0084,0.01,0.0087,-0.0156,-0.0274,-0.0110,NaN,NaN,3.000000e-10,5.500000e-09


#### Injection pressure linearity

In [10]:
## Checking injection volume pressure drop linearity
#defining a function for the straight line
def fit(x,a,b):
    return a*x + b

#curve fit for the data
y_val = vaclog["injection 1mbar"].values
x_val = (vaclog["Elapsed time"]/3600).values
params, cov = curve_fit(fit,x_val,y_val)

#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = -0.00*x+0.35


## Apparent desorption yield calculation for H2
### Formula for the cryo setup
$$
\eta_{app}=\frac{C_{comb}\cdot (p_{n,EM peak} - \overline{p}_{base}) \cdot q_e}{k_B\cdot T \cdot I_{n}}
$$

T here is @RT and therefore thermal transpiration compensation should be applied afterwards to $\Delta p$.

$C_{comb}$ takes into account the conductance from the sample via the copper tube for H2 and the pumping speed for turbo pump

In [11]:
#calculation of delta_p-s
dp1 = (np.max(em_h2)-np.max(base_h2))*100
print(dp1)

#Constants
k_B = scipy.constants.value("Boltzmann constant")
T = 4.2
q_e = scipy.constants.value("elementary charge")
c_h2 = 37.91/1000
s_h2=280/1000
c_comb=1/((1/c_h2)+(1/s_h2))
print(c_comb)


1.0170772114064996e-09
0.01992516581153798


In [12]:
#Calculating emission current
#create a new dataframe to filter emission current values 
#ignore the inrush current values 
emission = aligned_df[40:75].copy().reset_index(drop=True)

#emission["I_em"].where(emission["I_em"] <= 1.5e-6, np.nan, inplace=True)
print(emission.isnull().values.any())

#ignore low emission current (no-desorption condition)
emission["I_em"].where(emission["I_em"] >= 1e-7, np.nan,inplace=True)
print(emission)

#for calculating the maximum desorption yield:
#average the emission current during desorption
I_n = np.nanmean(emission["I_em"].nlargest(30))
print(I_n)



True
              mass 2.0_time  mass 2.0_value            mass 4.0_time  \
0   2022/12/13 17:38:31.708    2.040180e-11  2022/12/13 17:38:31.958   
1   2022/12/13 17:38:38.033    2.044220e-11  2022/12/13 17:38:38.283   
2   2022/12/13 17:38:44.358    2.052920e-11  2022/12/13 17:38:44.608   
3   2022/12/13 17:38:50.683    2.048840e-11  2022/12/13 17:38:50.933   
4   2022/12/13 17:38:57.008    2.134950e-11  2022/12/13 17:38:57.258   
5   2022/12/13 17:39:03.333    2.101780e-11  2022/12/13 17:39:03.583   
6   2022/12/13 17:39:09.658    2.086680e-11  2022/12/13 17:39:09.908   
7   2022/12/13 17:39:15.983    2.083240e-11  2022/12/13 17:39:16.233   
8   2022/12/13 17:39:22.308    2.088240e-11  2022/12/13 17:39:22.558   
9   2022/12/13 17:39:28.633    2.069480e-11  2022/12/13 17:39:28.883   
10  2022/12/13 17:39:34.958    2.078890e-11  2022/12/13 17:39:35.208   
11  2022/12/13 17:39:41.283    2.063780e-11  2022/12/13 17:39:41.533   
12  2022/12/13 17:39:47.608    2.075280e-11  2022/12/13 17:

In [13]:
#point-by-point desorption yield
#calculation of delta_p-s
p_deltas = (em_h2 - np.mean(base_h2))*100
p_deltas.clip(0, inplace=True)

#apply the desorption yield formula
tot_esd = (c_comb*p_deltas*q_e)/(k_B*T*emission["I_em"])
tot_esd.dropna(inplace=True)
print(tot_esd)

Series([], dtype: float64)


In [14]:
#maximum desorption yield
eta_h2 = (c_comb*dp1*q_e)/(k_B*T*I_n)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("S1 The maximum desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)

S1 The maximum desorption yield for H2 [mol/el]: 0.1490642221500728 



### Electron dose


Find:

$$
Q = \int_{t_1}^{t_2} I \,dt
$$

Where $t_2$ and $t_1$ are the timestamps at which an emission current reading was taken


In [15]:
#define the n, n-1 difference in a variable "t2"
t2 = emission["Elapsed time"].diff(periods=1).fillna(0)

#to avoid false integration times, replace all values that are further than 
#replace all values from t2 that are further away than 2x of stdev of mean
t2.where(t2 <= (t2.mean()+2*np.mean(t2)),t2.mean(),inplace=True)
t2.where(t2 >= (t2.mean()-2*np.mean(t2)),t2.mean(),inplace=True)

t2.dropna(inplace=True)
print(len(t2))

35


In [16]:
#alternative way: np.cumsum(current)*time
integral = np.cumsum(emission.I_em*t2)
print(integral)

0     0.000000
1     0.000015
2     0.000022
3     0.000025
4     0.000027
5     0.000029
6     0.000031
7     0.000033
8     0.000035
9     0.000038
10    0.000040
11    0.000042
12    0.000044
13    0.000046
14    0.000048
15    0.000050
16    0.000051
17    0.000053
18    0.000055
19    0.000056
20    0.000057
21    0.000059
22    0.000060
23    0.000061
24    0.000062
25    0.000063
26    0.000064
27    0.000065
28    0.000066
29    0.000067
30    0.000068
31    0.000069
32    0.000070
33    0.000071
34    0.000072
dtype: float64


In [17]:
#calculate the cumulative charge
Q_cumul = np.cumsum(integral)
Q_sum = sum(integral)
print("Total charge emitted from EGA during ~240 seconds:", f"{Q_sum:.4E}", "C")

#cumulative nr of electrons emitted
N_e = Q_sum/q_e
Ne_cumul = Q_cumul/q_e
print("Total number of electron emitted during ~240 seconds:", f"{N_e:.4E}", "e")
      
#To get the electron dose, divide n of electrons by sample surface area of 276 cm2, multiply by grid transmittance of 0.9
tot_dose = (N_e/276)*0.9
cumul_dose = (Ne_cumul/276)*0.9
print("Total electron dose on the sample ~240 seconds:", f"{tot_dose:.4E}", "e-/cm^2")
      
#add these to the "i" dataframe:
emission.insert(0,"Cumulative charge", Q_cumul)
emission.insert(1,"Cumulative n. of electrons emitted from EGA", Ne_cumul)
emission.insert(2,"Cumulative electron dose on the HL sample", cumul_dose)
emission.set_index("datetime", drop=True, inplace=True)


Total charge emitted from EGA during ~240 seconds: 1.7177E-03 C
Total number of electron emitted during ~240 seconds: 1.0721E+16 e
Total electron dose on the sample ~240 seconds: 3.4959E+13 e-/cm^2


## Plotting

In [18]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [19]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))

plt.plot(new_df["Timestamp, mass 2"],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(new_df["Timestamp, mass 4"],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(new_df["Timestamp, mass 12"],mid["mass 12.0_value"],marker=".", markersize=4,label='M12')
plt.plot(new_df["Timestamp, mass 15"],mid["mass 15.0_value"],marker=".", markersize=4,label='M16')
plt.plot(new_df["Timestamp, mass 16"],mid["mass 16.0_value"],marker=".", markersize=4,label='M15')
plt.plot(new_df["Timestamp, mass 18"],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(new_df["Timestamp, mass 28"],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(new_df["Timestamp, mass 32"],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(new_df["Timestamp, mass 40"],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
# plt.plot(new_df["Timestamp, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Timestamp')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("S1 MID scan")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(14))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

#write comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.95,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)


plt.savefig(r'./graphs/MID.png')
plt.show()


In [20]:
#plotting HiVolta log
plt.figure(figsize=(12,6))

plt.plot(hv["datetime"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["datetime"],hv["I_em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Timestamp')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("S1 HiVolta log")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.8,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)
    
plt.savefig(r'./graphs/HiVolta.png')
plt.show()


In [21]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S1 Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./graphs/Barions.png')
plt.show()

In [22]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S1 Temperature evolution")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/Temp.png')
plt.show()

In [23]:
#esd vs electron dose
plt.figure(figsize=(10,6))
plt.plot(emission["Cumulative electron dose on the HL sample"],integral,marker=".", markersize=4,label='eta')
#plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Electron dose (e/cm^2)')
plt.ylabel('ESD yield (Mol/el)')
plt.legend()
plt.title("Electron stimulated desorption yield vs electron dose")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.set_yscale('linear')
#ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

plt.savefig(r'./graphs/esd_vs_dose.png')
plt.show()

In [24]:
#checking inj pressure linearity, graph
fig, ax = plt.subplots(figsize=(10,6))
y_fit = fit(x_val,a,b)
ax.plot((vaclog["Time"]),vaclog["injection 1mbar"],marker=".", markersize=5,label='inj volume pressure')
ax.legend(loc="upper right")
ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')
ax.get_yaxis().get_major_formatter().set_useOffset(False)
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

ax2 = ax.twiny()
ax2.plot((vaclog["Elapsed time"]),y_fit, "--",color="red", label="Y= -ax + b fit") 
ax2.legend(loc="lower left")
ax2.set_xlabel('Elapsed time (s)')
ax2.set_ylabel('Pressure (mbar)')
ax2.set_yscale('linear')
ax2.get_yaxis().get_major_formatter().set_useOffset(False)

plt.title("S1 Injection pressure evolution 1mbar conductance gauge")
plt.savefig(r'./graphs/Injection.png')
plt.show()

In [25]:
print(emission["Cumulative electron dose on the HL sample"])
print(tot_esd)

datetime
2022-12-13 17:38:32    0.000000e+00
2022-12-13 17:38:38    3.076561e+11
2022-12-13 17:38:44    7.651553e+11
2022-12-13 17:38:52    1.283768e+12
2022-12-13 17:38:58    1.841999e+12
2022-12-13 17:39:04    2.440721e+12
2022-12-13 17:39:10    3.079187e+12
2022-12-13 17:39:16    3.758614e+12
2022-12-13 17:39:22    4.479251e+12
2022-12-13 17:39:29    5.250975e+12
2022-12-13 17:39:35    6.063940e+12
2022-12-13 17:39:41    6.917295e+12
2022-12-13 17:39:48    7.815847e+12
2022-12-13 17:39:54    8.750885e+12
2022-12-13 17:40:00    9.721170e+12
2022-12-13 17:40:07    1.073237e+13
2022-12-13 17:40:13    1.177551e+13
2022-12-13 17:40:19    1.285045e+13
2022-12-13 17:40:26    1.396183e+13
2022-12-13 17:40:32    1.510317e+13
2022-12-13 17:40:38    1.627317e+13
2022-12-13 17:40:46    1.747083e+13
2022-12-13 17:40:52    1.869083e+13
2022-12-13 17:40:58    1.993223e+13
2022-12-13 17:41:04    2.119512e+13
2022-12-13 17:41:10    2.247936e+13
2022-12-13 17:41:16    2.378371e+13
2022-12-13 17:41:23

## Writing edited dataframes to file

In [26]:
#new_df together with MID ion current values
#take rows 1,3,5,7,9.. from new_df and rows 2,4,6,8.. from MID
mid_signal = pd.DataFrame(mid.iloc[:,1::2])
comb = pd.concat([new_df, mid_signal], axis=1)
comb = comb[[item for items in zip(new_df.columns, mid_signal.columns) for item in items]]
comb.to_csv(r'./data analysis/MID_output.csv', sep='\t', encoding='utf-8', header=True)

#desorption yield params
esd_dict = {"p_H2 during ESD":em_h2, "H2 base pressure":base_h2, "conductance factor":c_comb,"Temp":T}
esd = pd.DataFrame(esd_dict)
esd.to_csv(r'./data analysis/esd_output.csv', sep='\t', encoding='utf-8', header=True)

#hivolta log
hv.to_csv(r'./data analysis/HV_output.csv', sep='\t', encoding='utf-8', header=True)

#vaclog
vaclog.to_csv(r'./data analysis/vaclogger_output.csv', sep='\t', encoding='utf-8', header=True)

#CERNoX
temps_dict = {"Timestamp":vaclog["Time"],"Temperatures":polyfit(A,vaclog["T-CERNOX"])}
temps = pd.DataFrame(temps_dict)
temps.to_csv(r'./data analysis/CERNoX temps.csv', sep='\t', encoding='utf-8', header=True)

#electron dose
emission.to_csv(r'./data analysis/electron dose calculations.csv', sep='\t', encoding='utf-8', header=True)
